In [4]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base

In [5]:
# create the engine and session
engine = create_engine("sqlite:///c_three_tables.db")
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
Base = declarative_base()

### define the models

In [7]:
class Customer(Base):
    __tablename__ = "customers"

    id = Column(Integer, primary_key=True)
    name = Column(String)

    orders = relationship("Order", back_populates="customer")

In [8]:
class Order(Base):
    __tablename__ = "orders"

    id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey("customers.id"))

    customer = relationship("Customer", back_populates="orders")
    items = relationship("OrderItem", back_populates="order")

In [9]:
class OrderItem(Base):
    __tablename__ = "order_items"

    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey("orders.id"))
    product_name = Column(String)
    quantity = Column(Integer)

    order = relationship("Order", back_populates="items")

In [10]:
# create the tables
Base.metadata.create_all(engine)

In [11]:
# create some data
customer1 = Customer(name="John")
customer2 = Customer(name="Jane")

session.add_all([customer1, customer2])
session.commit()

In [12]:
order1 = Order(customer=customer1)
order2 = Order(customer=customer2)

session.add_all([order1, order2])
session.commit()

In [13]:
item1 = OrderItem(order=order1, product_name="Product A", quantity=3)
item2 = OrderItem(order=order2, product_name="Product B", quantity=1)

session.add_all([item1, item2])
session.commit()

In [14]:
# join two tables (orders and customers)
query = session.query(Order, Customer).join(Customer)
for order, customer in query:
    print(f"Order {order.id} belongs to customer {customer.name}")

Order 1 belongs to customer John
Order 2 belongs to customer Jane


In [15]:
# join three tables (orders, customers, and order_items)
query = session.query(Order, Customer, OrderItem).join(Customer).join(OrderItem)
for order, customer, item in query:
    print(
        f"Customer {customer.name} ordered {item.quantity} of {item.product_name} in order {order.id}"
    )

Customer John ordered 3 of Product A in order 1
Customer Jane ordered 1 of Product B in order 2
